In [ ]:
import add_packages
import config
from pprint import pprint
import os, re, unicodedata, string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from my_langchain import (
  document_loaders, text_splitters, text_embedding_models, vector_stores, 
  chat_models, prompts, utils, output_parsers, agents, documents
)

# Data

## Lectures Content

### Combine

In [ ]:
def extract_course_name(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        first_line = file.readline().strip()
        return first_line[len("# Course: "):]

In [ ]:
def remove_extra_newlines(text):
    while '\n\n' in text:
        text = text.replace('\n\n', '\n')
    return text


def remove_double_spaces(text):
    while '  ' in text:
        text = text.replace('  ', ' ')
    return text


def process_add_space_after_hash(text):
    pattern = r'##(\w)'
    processed_text = re.sub(pattern, r'## \1', text)
    return processed_text


def process_remove_quotes(text):
    cleaned_text = text.replace('"', '')
    return cleaned_text


def process_text_file(input_file, output_file, functions):
    # Read the first line of the input file
    with open(input_file, 'r') as file:
        first_line = file.readline().strip()
        len_first_line = len(first_line)
    

    # Extract course name from filename
    course_name = os.path.splitext(os.path.basename(input_file))[0]
    # If the first line doesn't match the filename, replace it with the filename's course name
    first_line = "# Course: " + course_name

    # Reopen the file to process the entire content
    with open(input_file, 'r') as file:
        text = file.read()

    # Apply processing functions
    for func in functions:
        text = func(text)

    # Prepend the modified first line to the processed text
    text = first_line + '\n' + text[len_first_line + 1:]

    # Write the processed text to the output file
    with open(output_file, 'w') as file:
        file.write(text)


# Define the list of functions to apply
functions_to_apply = [
    process_remove_quotes,
    remove_extra_newlines,
    remove_double_spaces,
    process_add_space_after_hash,
]

# Example usage:
input_file_path = '../data/vtc1/script/KỸ NĂNG TÌM VIỆC LÀM THÊM.txt'
output_file_path = '../data/vtc1/script/KỸ NĂNG TÌM VIỆC LÀM THÊM.txt'
process_text_file(input_file_path, output_file_path, functions_to_apply)

In [ ]:
# Specify the folder containing the files
folder_path = '../data/vtc1/script/'

# Loop through each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.txt'):  # Process only text files
        input_file_path = os.path.join(folder_path, filename)
        output_file_path = os.path.join(
            folder_path, filename)  # Output file path
        process_text_file(input_file_path, output_file_path,
                          functions_to_apply)

In [ ]:
# Create documents of one course
# Example usage:
file_path = "../data/vtc1/script/HỘI HỌA - MÀU SẮC VÀ PHỐI MÀU CƠ BẢN.txt"
course_name = extract_course_name(file_path).lower()

text_loader_lectures_content = document_loaders.TextLoader(file_path)
document = text_loader_lectures_content.load()

text_splitter = text_splitters.RecursiveCharacterTextSplitter(
  chunk_size=1000, chunk_overlap=200,
)
docs_lectures_content = text_splitter.split_documents(document)

metadatas = {
  "data": "lectures content",
  "course_name": course_name,
}
utils.remove_metadata(docs_lectures_content, "source")
utils.update_metadata(docs_lectures_content, metadatas)

In [ ]:
def process_folder(folder_path):
    data_dict = {}
    for root, dirs, files in os.walk(folder_path):
        for file_name in files:
            if file_name.endswith('.txt'):
                file_path = os.path.join(root, file_name)
                course_name = extract_course_name(file_path).lower()

                text_loader_lectures_content = document_loaders.TextLoader(file_path)
                document = text_loader_lectures_content.load()

                text_splitter = text_splitters.RecursiveCharacterTextSplitter(
                    chunk_size=1000, chunk_overlap=200,
                )
                docs_lectures_content = text_splitter.split_documents(document)

                metadatas = {
                    "data": "lectures content",
                    "course_name": course_name,
                }
                utils.remove_metadata(docs_lectures_content, "source")
                utils.update_metadata(docs_lectures_content, metadatas)

                data_dict[course_name] = docs_lectures_content

    return data_dict


folder_path = "../data/vtc1/script/"
docs_lectures_content_dict = process_folder(folder_path)
for file_name, content in docs_lectures_content_dict.items():
    print(f"File: {file_name}, Course: {content[0].metadata['course_name']}")

## FAQ

In [ ]:
text_loader_faq = document_loaders.TextLoader(
  "../data/vtc/faq.txt"
)
document = text_loader_faq.load()

text_splitter = text_splitters.RecursiveCharacterTextSplitter(
  chunk_size=500, chunk_overlap=100,
)
docs_faq = text_splitter.split_documents(document)

metadatas = {
  "data": "frequently asked questions"
}
utils.remove_metadata(docs_faq, "source")
utils.update_metadata(docs_faq, metadatas)

## Courses list

In [ ]:
courses_list_cols = utils.get_csv_column_names("../data/vtc1/courses_list.csv")

csv_loader_courses_list = document_loaders.CSVLoader(
  "../data/vtc1/courses_list.csv",
  # source_column="No",
  csv_args={
      "delimiter": ",",
      # "quotechar": "''",
      "fieldnames": courses_list_cols,
  },
)
document = csv_loader_courses_list.load()[1:]
docs_courses_information = document

metadatas = {
  "data": "courses information"
}
utils.remove_metadata(docs_courses_information, "source")
utils.remove_metadata(docs_courses_information, "row")
utils.update_metadata(docs_courses_information, metadatas)

# Vector store 

In [ ]:
qdrant_instance_lectures_content = vector_stores.QdrantWrapper(
  collection_name="vtc-lectures-content-new",
  qdrant_host=os.getenv("QDRANT_HOST"),
  qdrant_api_key=os.getenv("QDRANT_API_KEY"),
  default_search_type="similarity",
  default_search_kwargs={"k": 5},
  retriever_tool_name="lectures_content",
  retriever_tool_description="Searches and returns content, knowledge from \
    the lecture scripts based on specialized keywords from user's question like \
    Typography, Lazada, Premiere, Unity ...",
)

In [ ]:
# # Add multiple courses
# for docs in docs_lectures_content_dict.values():
#   qdrant_instance_lectures_content.add_documents(docs)

In [ ]:
# Add one course
# qdrant_instance_lectures_content.add_documents(docs_lectures_content)

In [ ]:
qdrant_instance_faq = vector_stores.QdrantWrapper(
  collection_name="vtc-faq",
  qdrant_host=os.getenv("QDRANT_HOST"),
  qdrant_api_key=os.getenv("QDRANT_API_KEY"),
  default_search_type="similarity",
  default_search_kwargs={"k": 10},
  retriever_tool_name="frequently_asked_questions",
  retriever_tool_description="Searches and returns answer for frequently asked \
    questions about Onlinica information like accounts, fees, courses, payments, \
    certificates ...",
)

In [ ]:
# qdrant_instance_faq.add_documents(docs_faq)

In [ ]:
qdrant_instance_courses_information = vector_stores.QdrantWrapper(
    collection_name="vtc-courses-information-new",
    qdrant_host=os.getenv("QDRANT_HOST"),
    qdrant_api_key=os.getenv("QDRANT_API_KEY"),
    default_search_type="similarity",
    default_search_kwargs={"k": 10},
    retriever_tool_name="courses_information",
    retriever_tool_description="Searches and returns information about courses \
      of Onlinica like course name, course category, course link, course \
      description, total number of courses ...",
)

In [ ]:
# qdrant_instance_courses_information.add_documents(docs_courses_information)

# Tools

In [ ]:
"""
Bạn là một nhân viên tư vấn của một nền tảng học tập trực tuyến tên là Onlinica.

Bạn có các phẩm chất sau: 
- Hay giúp đỡ
- Cực kỳ tận tâm và chăm chỉ
- Tư cách chuyên nghiệp, tôn trọng, chân thành và trung thực
- Tư duy chuẩn mực
- Kỹ năng giao tiếp, đàm phán, xử lý phàn nàn xuất sắc
- Kỹ năng bán hàng xuất sắc
- Hiểu biết sâu sắc về sản phẩm/dịch vụ. Kiến thức vững về ngành nghề
- Tinh thần lạc quan và tích cực. Khả năng tạo ra trải nghiệm tích cực cho khách hàng
- Nhạy bén với yêu cầu và mong muốn của khách hàng

Bạn sẽ giúp người dùng trả lời các câu hỏi về các khóa học trên nền tảng. Ngôn ngữ mà bạn trả lời sẽ giống với ngôn ngữ của người dùng.

Các câu hỏi mà người dùng có thể hỏi và cách trả lời:
- Thông tin cơ bản về khóa học: Bạn NÊN liệt kê TẤT CẢ các khóa học hiện có (và thông tin của chúng) LIÊN QUAN đến câu hỏi của người dùng.
- Nội dung, kiến thức của bài giảng: Các câu hỏi này thường sẽ chứa các từ khóa chuyên ngành như Typography, Lazada, Premiere, Unity,... Bạn sẽ tổng hợp thông tin từ script của các bài giảng có chứa từ khóa chuyên ngành đó và đưa ra câu trả lời chi tiết cho người dùng. Bạn nên gợi ý các khóa học (tên khóa học kèm link khóa học) có liên quan đến từ khóa chuyên ngành đó cho người dùng.
- Các câu hỏi thường được hỏi
"""

onlinica_system_message = """\
You are a consultant for an online learning platform called Onlinica.

You have the following qualities:
- Helpful
- Extremely dedicated and hardworking
- Professionalism, respect, sincerity and honesty
- Standard thinking
- Excellent communication, negotiation and complaint handling skills
- Excellent sales skills
- Deep understanding of products/services. Strong knowledge of the industry
- Optimistic and positive spirit. Ability to create a positive customer experience
- Sensitive to customers' requests and desires

You will help users answer questions about the courses on the platform. The language in which you respond will be the same as the user's language.

Questions users might ask and how to answer:
- Course basics: You SHOULD list ALL available courses (and their information) that are RELEVANT to the user's question.
- Content and knowledge of the lecture: These questions will often contain specialized keywords such as Typography, Lazada, Premiere, Unity,... You will synthesize information from the scripts of the lectures that contain keywords that major and give detailed answers to users. You should suggest courses (course name with course link) related to that specialized keyword to users.
- Frequently asked questions\
"""
onlinica_prompt = prompts.create_openai_tools_agent_custom_prompt(onlinica_system_message)

In [ ]:
tools = [
  qdrant_instance_lectures_content.retriever_tool,
  qdrant_instance_faq.retriever_tool,
  qdrant_instance_courses_information.retriever_tool,
]

llm = chat_models.chat_openai
agent = agents.MyAgent(prompt=onlinica_prompt, tools=tools, agent_type="openai_tools", llm=llm)

In [ ]:
questions = [
  "xin chào. Tên tôi là Bob.",
  "bạn có nhớ tên tôi là gì không",
  
  "digital marketing là gì",
  
  "làm cách nào để đăng ký tài khoản onlinica",
  "có mấy loại tài khoản onlinica",
  "các khoá học tại onlinica có thời hạn sử dụng bao lâu",
  "onlinica có mấy hình thức thanh toán",
  "có thể thanh toán bằng momo được không",
  
  "các khóa học về design",
  "các khóa học về trí tuệ nhân tạo",
  "các khóa học về  ai",
  "các khóa học của nguyễn ngọc tú uyên",
  "các khóa học của tú uyên",
  "các khóa học thầy trần anh tuấn dạy",
  
  "cách quản lý thời gian",
  "nguyên lý phối màu",
]

In [ ]:
response = agent.invoke_agent(
    "có thể thanh toán bằng momo được không"
)
pprint(response)